In [18]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun 
import numpy as np
from pyspark.sql.types import *
from collections import Counter
from pyspark.sql import functions as F


In [19]:
spark = SparkSession.builder.getOrCreate()

In [20]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [21]:
nums = np.arange(1, 50)

rdd = sc.parallelize(nums.tolist())
print(rdd.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


Find the sum, average, maximum, minimum, and count

In [22]:
#Sum
print("Sum:", rdd.sum())




Sum: 1225


In [23]:
#Avgerage
print("Average:", rdd.mean())

Average: 25.0


In [24]:
#Count
print("Count:", rdd.count())


Count: 49


In [25]:
#Min
print("Min:", rdd.min())

Min: 1


In [26]:
#Max
print("Max:", rdd.max())

Max: 49


Count how many numbers are even vs. odd.

In [27]:
odd_count = rdd.filter(lambda x: x % 2 != 0).count()
even_count = rdd.filter(lambda x: x % 2 == 0).count()
print("ODD Numbers:", odd_count, "EVEN Numbers:", even_count)

ODD Numbers: 25 EVEN Numbers: 24


You have the following data of people info ('Name', 'Age'), answer the following questions

In [28]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [29]:
oldest = rdd_people.max(lambda x: x[1])
print(oldest)

('Khaled', 40)


Compute the average age

In [30]:
avg_age = rdd_people.map(lambda x: x[1]).mean() 
print(avg_age)

31.666666666666668


Group all the names by their age

In [31]:
age_grouped = rdd_people.groupBy(lambda x: x[1]).map(lambda x: (x[0], list(x[1])))
print(age_grouped.collect())

[(35, [('Ahmed', 35), ('Ahmed', 35)]), (40, [('Khaled', 40)]), (25, [('Nada ', 25), ('Nada ', 25)]), (30, [('Mona', 30)])]


Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [32]:
rdd_russia = sc.textFile("/home/jovyan/work/russia.txt")
print(rdd_russia.collect())


['Russia is the largest country in the world by land area', 'Moscow is the capital city of Russia', 'The Russian language is one of the most widely spoken languages in the world', 'Russia is known for its rich history and culture', 'The Trans-Siberian Railway is the longest railway line in the world', 'Russia has a strong tradition in literature, music and ballet', 'The country is famous for its cold winters and vast landscapes', 'Russia is a major player in global energy production']


Count the total number of lines.

In [33]:
line_count = rdd_russia.count()
print(line_count)

8


Count how many lines contain the word "Russia"

In [34]:
russia_lines = rdd_russia.filter(lambda line: "Russia" in line).count()
print(russia_lines)

6


Find the most 5 frequent word in the file.

In [35]:
top_words = (rdd_russia
.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)           
.sortBy(lambda x: x[1], ascending=False).take(5)                                     
)

print(top_words)


[('is', 7), ('the', 7), ('Russia', 5), ('in', 5), ('world', 3)]


Tokenize words

In [36]:
tokens = (rdd_russia.flatMap(lambda line: line.split()).collect())                        
print(tokens)

['Russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'Moscow', 'is', 'the', 'capital', 'city', 'of', 'Russia', 'The', 'Russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'Russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'The', 'Trans-Siberian', 'Railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'Russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', 'The', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'Russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production']


Remove stopwords (a, the, is, to, in, of). 

In [37]:
stopwords = {"a", "the", "is", "to", "in", "of"}

filtered_tokens = [word for word in tokens if word not in stopwords]
print(filtered_tokens)

['Russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'Moscow', 'capital', 'city', 'Russia', 'The', 'Russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'Russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'The', 'Trans-Siberian', 'Railway', 'longest', 'railway', 'line', 'world', 'Russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', 'The', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'Russia', 'major', 'player', 'global', 'energy', 'production']


Count the frequency of each word

In [38]:
words_count = Counter(filtered_tokens)   
print(list(words_count.items()))


[('Russia', 5), ('largest', 1), ('country', 2), ('world', 3), ('by', 1), ('land', 1), ('area', 1), ('Moscow', 1), ('capital', 1), ('city', 1), ('The', 3), ('Russian', 1), ('language', 1), ('one', 1), ('most', 1), ('widely', 1), ('spoken', 1), ('languages', 1), ('known', 1), ('for', 2), ('its', 2), ('rich', 1), ('history', 1), ('and', 3), ('culture', 1), ('Trans-Siberian', 1), ('Railway', 1), ('longest', 1), ('railway', 1), ('line', 1), ('has', 1), ('strong', 1), ('tradition', 1), ('literature,', 1), ('music', 1), ('ballet', 1), ('famous', 1), ('cold', 1), ('winters', 1), ('vast', 1), ('landscapes', 1), ('major', 1), ('player', 1), ('global', 1), ('energy', 1), ('production', 1)]


In [39]:
schema = 'id integer, name string, age integer, salary integer' 
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [40]:
df.printSchema()
df.show(2)


root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [41]:
df.select("name", "salary").show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [42]:
df.select(F.avg("salary").alias("average salary")).show()


+--------------+
|average salary|
+--------------+
|        6000.0|
+--------------+



Filter employees older than 28

In [ ]:
df.filter(df.age > 28).show()


+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [ ]:
df.select("name").distinct().count()

4

Group by a the name column and find average salary

In [47]:
df.groupBy("name").agg(F.avg("salary").alias("average_salary")).show()

+------+--------------+
|  name|average_salary|
+------+--------------+
|   Ali|        4000.0|
|Mariam|        6750.0|
|  Omar|        6750.0|
|  Sara|        5000.0|
+------+--------------+



In [48]:
df1 = spark.read.csv("/home/jovyan/work/NullData.csv", header=True, inferSchema=True) #this file in shared folder
df1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Find the avg sales 

In [49]:

avg_sales = df1.select(F.avg("Sales").alias("average_sales")).collect()[0][0]
print(avg_sales)

400.5


Replace null name with 'Unknown' and sales with the avg sales of the column 

In [50]:
df_filled = df1.fillna({"Name": "Unknown", "Sales": avg_sales})

df_filled.show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|400.5|
|emp2|Unknown|400.5|
|emp3|Unknown|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

